Install Jupyter Notebook (if not installed)

If you haven't installed Jupyter yet, you can install it using pip:


    pip install notebook

In [1]:
import setup

setup.init_django()



c:\Users\Nazbeen Ai\Documents\My Websites\Nazbeen_Online_PlatForm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from courses.models import Course

In [18]:
# Fetch all courses with necessary fields (title, description, etc.)
courses = Course.objects.all().values('title', 'description', 'instructor__user', 'subject__title')

# Prepare data for fine-tuning (for example, using T5 for text-to-text retrieval)
train_data = []

for course in courses:
    query = "What is this course about?"
    course_desc = course['description']
    train_data.append({
        'input_text': query,
        'output_text': course_desc
    })

print(train_data)



[{'input_text': 'What is this course about?', 'output_text': 'Understand the fundamentals of project management, including Agile and Scrum methodologies.'}, {'input_text': 'What is this course about?', 'output_text': 'Learn to manage personal finances, budget effectively, and invest wisely.'}, {'input_text': 'What is this course about?', 'output_text': 'Learn to manage personal finances, budget effectively, and invest wisely.'}, {'input_text': 'What is this course about?', 'output_text': 'Create immersive VR games using Unity and C#.'}, {'input_text': 'What is this course about?', 'output_text': 'Understand user interface and user experience design principles, prototyping, and usability testing.'}, {'input_text': 'What is this course about?', 'output_text': 'Optimize websites for better search engine rankings.'}, {'input_text': 'What is this course about?', 'output_text': 'Create beautiful digital art using Procreate.'}, {'input_text': 'What is this course about?', 'output_text': 'Unde

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# Load the pre-trained T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Prepare dataset for training (example list of input-output pairs)

# Convert training data into Hugging Face Dataset
train_dataset = Dataset.from_dict({
    "input_text": [item['input_text'] for item in train_data],
    "output_text": [item['output_text'] for item in train_data],
})

# Tokenization function
def preprocess_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output_text"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset
train_dataset = train_dataset.map(preprocess_function, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",  # Disable evaluation
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=10,  # Save checkpoint every 10 steps
    logging_dir='./logs',  # Log directory
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


Map:   0%|          | 0/304 [00:00<?, ? examples/s]c:\Users\Nazbeen Ai\Documents\My Websites\Nazbeen_Online_PlatForm\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 304/304 [00:02<00:00, 120.66 examples/s]
c:\Users\Nazbeen Ai\Documents\My Websites\Nazbeen_Online_PlatForm\.venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Nazbeen Ai\AppData\Local\Temp\ipykernel_11952\2447169141.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Tra

In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load your fine-tuned model
model = T5ForConditionalGeneration.from_pretrained('results/checkpoint-3')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def get_course_response(query):
    # Prepare the input query
    input_text = f"What is this course about? {query}"
    
    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    # Generate the model output
    output = model.generate(inputs['input_ids'], max_length=200, num_beams=5, early_stopping=True)
    
    # Decode the output to get the course description
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return response


In [12]:
input_query = "Fine Turning"

print(get_course_response(input_query))

Fine Turning
